In [1]:
#Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys
import os
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import tqdm
from tqdm.notebook import tqdm
import re

#드라이버 설치
https://chromedriver.chromium.org/downloads
#드라이버랑 크롬버젼이랑 일치시켜주어야함. 드라이버 다운 받을 때 참고(메뉴 => 도움말 => Chrome 정보에서 확인)

In [2]:
#Step 1. 크롬 웹브라우저 실행
#경로설정
path = "C:/Users/bae/Desktop/chromedriver.exe"   # 맥은 "chromedriver"
driver = webdriver.Chrome(path)

In [3]:
txt_file_path = 'productUrlList.txt'

In [4]:
#productUrlList.txt에서 링크 리스트 불러오기
r = open(txt_file_path, mode='rt', encoding='utf-8')
productUrlList = r.readlines()
len(productUrlList)

633

In [5]:
import json

def toJson(prod_data, file_path) :
    with open(file_path, 'w', encoding='utf-8') as file :
        json.dump(prod_data, file)
    return 
  
file_path = 'prod_data.json'

error_ls = []
prod_data = []

for prod_url in tqdm(productUrlList[:10]) :
    try :
        driver.get(prod_url)
        time.sleep(1)

        #상품 정보 뽑기

        #상품 고유번호
        prod_idx = re.sub('http://www.tmon.co.kr/deal/','',prod_url)
        prod_idx = re.sub('\n','',prod_idx)
        prod_idx = int(prod_idx)

        #상품명
        productTitle= driver.find_element_by_class_name("deal_title_main").text

        #상품 카테고리 최대 6층
        category_ls = []
        for i in range(1, 7) :
            try :
                category= driver.find_element_by_class_name("category._depth{}".format(i)).text
                category_ls.append(category)
            except :
                break

        #상품 가격
        productPrice = driver.find_element_by_class_name("number_unit").text #'21,900원'으로 나오기 때문에 전처리 필요 정규표현식으로 숫자만 남기고 int
        temp = re.findall('[0-9]*', productPrice)
        productPrice = int(''.join(temp))

        #구매횟수 
        buy_count = driver.find_element_by_class_name("deal_price_buy_count").text
        temp = re.findall('[0-9]*', buy_count)
        buy_count = int(''.join(temp))

        #상품 만족도 star_rate_num
        try :
            prod_satis = float(driver.find_element_by_class_name("grade_average_score").text)
        except :
            prod_satis = 0.

        #상품이미지 url
        image = driver.find_element_by_class_name("active").find_element_by_tag_name('img').get_attribute('src')

        #상품종료여부(품절 = 0, 구매가능 = 1)
        if driver.find_elements_by_class_name("col2")[1].text == '바로구매' :
            stocks = 1
        else : 
            stocks = 0
        
        prod_data.append({
            'prod_idx' : prod_idx,
            'image' : image,
            'title' : productTitle,
            'category' : category_ls,
            'price' : productPrice,
            'stocks' : stocks,
            'prod_satis' : prod_satis,
            'prod_url' : prod_url,
            'buy_count' : buy_count
        })

    except : 
        error_ls.append(prod_url)

toJson(prod_data, file_path)
toJson(error_ls, 'error_data.json')

#중간에 멈췄으면, 아래 코드 실행 후 **반복문 범위와 **파일저장명(file_path) 변경해서 재실행
#print(productUrlList.index(prod_data[-1]))

In [6]:
data_df = pd.read_json('prod_data.json')
data_df

,prod_idx,image,title,category,price,stocks,prod_satis,prod_url,buy_count
0,4422624974,http://th3.tmon.kr/thumbs/image/a8a/5eb/089/fd...,[퍼스트위크] 필수특가 개당993원 신세계푸드 올반 치즈핫도그 15봉 / 풍미가득 ...,"[식품·건강, 간편식·냉장·냉동, 소시지·핫도그·피자·어묵, 핫도그]",14900,1,4.8,http://www.tmon.co.kr/deal/4422624974\n,596
1,3223462394,http://th2.tmon.kr/thumbs/image/cbf/ebd/f0a/1e...,[퍼스트위크] 티몬블랙딜 오쿡 닭가슴살 1+1+1+1 외 33종 2900원 균일가 ...,"[식품·건강, 간편식·냉장·냉동, 닭가슴살·오리훈제, 닭가슴살]",2900,1,4.6,http://www.tmon.co.kr/deal/3223462394\n,367954
2,4377275322,http://th4.tmon.kr/thumbs/image/dba/13c/4bb/00...,천하장사 콰트로치즈/콘스윗치즈 1+1+1,"[식품·건강, 간편식·냉장·냉동, 소시지·핫도그·피자·어묵, 소시지]",8900,1,0.0,http://www.tmon.co.kr/deal/4377275322\n,354
3,4422654646,http://th1.tmon.kr/thumbs/image/bc6/e46/a9c/80...,팔백도파막창 220g외 3종,"[식품·건강, 간편식·냉장·냉동, 양념·육가공·수산, 막창]",4900,0,0.0,http://www.tmon.co.kr/deal/4422654646\n,1438
4,4264247790,http://th1.tmon.kr/thumbs/image/953/892/3dd/21...,[퍼스트위크] 필수특가 삼시세떡 국물떡볶이 320g 외 야끼만두 김말이,"[식품·건강, 간편식·냉장·냉동, 떡볶이·면·떡·기타, 떡볶이]",1900,1,4.3,http://www.tmon.co.kr/deal/4264247790\n,25305
5,4069161210,http://th1.tmon.kr/thumbs/image/7d2/6ec/452/4d...,[퍼스트위크] 티몬블랙딜 통통떡볶이1-2인분 외 순대/튀김/어묵/핫바 모음,"[식품·건강, 간편식·냉장·냉동, 떡볶이·면·떡·기타, 떡볶이]",1900,1,4.4,http://www.tmon.co.kr/deal/4069161210\n,23025
6,4391405654,http://th3.tmon.kr/thumbs/image/3b2/1a6/16d/6d...,[퍼스트위크] 무한타임 20년 전통 거제 옛날 떡볶이 사계절 떡볶이 570g / 2...,"[식품·건강, 간편식·냉장·냉동, 떡볶이·면·떡·기타, 떡볶이]",2900,0,0.0,http://www.tmon.co.kr/deal/4391405654\n,1000
7,4427550370,http://th2.tmon.kr/thumbs/image/10b/d79/553/c7...,신세계 올반 크런치 찰핫도그 75gX10봉 외 인생왕교자/볶음밥/튀김/소불고기/ 구...,"[식품·건강, 간편식·냉장·냉동, 소시지·핫도그·피자·어묵, 핫도그]",9900,0,0.0,http://www.tmon.co.kr/deal/4427550370\n,168
8,4256216794,http://img2.tmon.kr/cdn3/deals/2020/08/28/4256...,"11+1 맛남주의 맛난식탁도시락 12종, 내마음대로 골라담기","[식품·건강, 간편식·냉장·냉동, 볶음밥·죽·간편식, 볶음밥]",2500,1,4.3,http://www.tmon.co.kr/deal/4256216794\n,37893
9,4422693502,http://th1.tmon.kr/thumbs/image/c3c/911/202/06...,백순대 강추/신림동 또순이 백순대 양념순대곱창볶음 1kg 4종,"[식품·건강, 간편식·냉장·냉동, 양념·육가공·수산, 순대]",9900,1,0.0,http://www.tmon.co.kr/deal/4422693502\n,286
